In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from __future__ import annotations

import jax.numpy as jnp
import jax
from pathlib import Path
from diffwake.diffwake_jax.model_agnostic import load_input, create_state, turbine_powers, alter_yaw_angles
from diffwake.diffwake_jax.sim_agnostic import simulate
from diffwake.diffwake_jax.turbine.operation_models import power as power_fn
from floris import FlorisModel, TimeSeries
import numpy as np
import time


# helper function
def absolute_percentage_difference(a, b):
    return jnp.abs((a - b) / b) * 100

## DiffWake yawed vs unyawed comparisons

In [11]:
# 1. Define paths to configuration files
data_dir = Path("data/horn")
farm_cfg_path = data_dir / "gauss_hornsRev.yaml"
turbine_cfg_path = data_dir / "vestas_v802MW.yaml"

# 2. Load the base configuration
cfg = load_input(farm_cfg_path, turbine_cfg_path)

# 3. Define a simple 3-turbine layout (aligned along the X-axis)
ws = 13.5
wd = 270.0
ti_val = 0.06
layout_x = jnp.array([0.0, 400.0, 800.0])
layout_y = jnp.array([0.0, 0.0, 0.0])

# 4. Define flow conditions
wind_speed = jnp.array([ws])
wind_dir = jnp.array([jnp.deg2rad(wd)]) # Wind from West
ti = jnp.array([ti_val])

# 5. Update the configuration
cfg = cfg.set(
    layout_x=layout_x,
    layout_y=layout_y,
    wind_speeds=wind_speed,
    wind_directions=wind_dir,
    turbulence_intensities=ti
)

# 6. Create the simulation state
state = create_state(cfg)

# --- Baseline: Unyawed Farm ---
yaw_angles_unyawed = jnp.zeros((1, 3)) # (n_findex, n_turbines)
state_unyawed = alter_yaw_angles(yaw_angles_unyawed, state)
result_unyawed = simulate(state_unyawed)
unyawed_turbine_powers = power_fn(power_thrust_table=state.farm.power_thrust_table,
                                  velocities=result_unyawed.u_sorted,
                                  air_density=state.flow.air_density,
                                  yaw_angles=state.farm.yaw_angles,)
unyawed_farm_power = jnp.sum(unyawed_turbine_powers)/1e6 # convert to megawatts

# state_final_unyawed = state_unyawed.replace(flow=state_unyawed.flow.replace(u=result_unyawed.u_sorted))
# powers_unyawed = turbine_powers(state_final_unyawed)[0] # First (and only) wind condition

# # --- Case: Yawed Farm ---
y1, y2, y3 = 20.0, 10.0, 0.0
yaw_angles_deg = jnp.array([[y1, y2, y3]]) # Try steering wakes away from downstream turbines
yaw_angles_rad = jnp.deg2rad(yaw_angles_deg)

state_yawed = alter_yaw_angles(yaw_angles_rad, state)

result_yawed = simulate(state_yawed)
yawed_turbine_powers = power_fn(power_thrust_table=state.farm.power_thrust_table,
                                  velocities=result_yawed.u_sorted,
                                  air_density=state.flow.air_density,
                                  yaw_angles=state.farm.yaw_angles,)
yawed_farm_power = jnp.sum(yawed_turbine_powers)/1e6

## FLORIS yaw

In [12]:
fmodel = FlorisModel(r"../floris/examples/inputs/gauss_vesta.yaml")

floris_yaw_angles = np.array([y1, y2, y3]).reshape(1, -1)
time_series = TimeSeries(
    wind_directions=np.array([wd]),
    wind_speeds=np.array([ws]),
    turbulence_intensities=ti_val,
)

fmodel.set(wind_data=time_series,
           layout_x=np.array([0.0, 400.0, 800.0]),
           layout_y=np.array([0.0, 0.0, 0.0]),
           yaw_angles=np.zeros_like(floris_yaw_angles))
fmodel.run()

# baseline case
floris_baseline_power = fmodel.get_farm_power() / 1e6
turbine_powers_floris = fmodel.get_turbine_powers()

# yawed case
fmodel.set(yaw_angles=floris_yaw_angles)
fmodel.run()
floris_yawed_power = fmodel.get_farm_power() / 1e6

In [13]:
print(f"----- FLORIS results -----")
print(f"Baseline farm power: {floris_baseline_power[0]:.6f} MW")
print(f"Yawed farm power: {floris_yawed_power[0]:.6f} MW\n")

print(f"----- DiffWake results -----")
print(f"Baseline farm power: {unyawed_farm_power:.6f} MW")
print(f"Yawed farm power: {yawed_farm_power:.6f} MW")

print("\n---------- Differences ----------")
print(f"Absolute baseline power difference (GCH) = {absolute_percentage_difference(floris_baseline_power, unyawed_farm_power)[0]:.4f}%")
print(f"Absolute yawed power difference (GCH) = {absolute_percentage_difference(floris_yawed_power, yawed_farm_power)[0]:.4f}%")

----- FLORIS results -----
Baseline farm power: 4.836709 MW
Yawed farm power: 5.102339 MW

----- DiffWake results -----
Baseline farm power: 4.836627 MW
Yawed farm power: 5.261699 MW

---------- Differences ----------
Absolute baseline power difference (GCH) = 0.0017%
Absolute yawed power difference (GCH) = 3.0287%
